# Analysis of U.S. Labor Demographics (1972-2015)
Braeden Wright | April 2022

------
# Project Goal

Analyze the U.S. Department of Labor statistics data for the U.S. labor force. Looking at the data, I want to try and predict future employment/ unemployment rate based on previous years data, if possible with the data available. Barring that, analyzing the data may also give insight into what populations have been affected the most when economic With the data being sorted within a timeframe, it will be an excellent opportunity to continue honing my time series analysis skills.

If any other notable trends appear they will certainly be investigated, and suggestions on how to better the dataframe or what other features would be helpful for analysis.

------
# Planning
- Setup github repository and environment (.gitignore, README.md)
- Import data from source (.csv)
- Prep data for exploration (rename, feature engineer)
- Explore questions about the data
- Visually represent findings
- Report conclusions and recommendations based on data analysis

-------
# Executive summary
***Key Takeaways***
- 
-

***Recommendations***
-
-

***Next Steps***
-
-

---------------



## Import Menagerie

In [1]:
# Basics
import numpy as np
import pandas as pd
import scipy.stats as stats
from pydataset import data
from scipy import math
import datetime
from datetime import datetime

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Models and Metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Custom
import wrangle

# Make sure we can see the full scale of the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# plotting defaults
plt.rc('figure', figsize=(11, 5))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

# Data Aquisition and Preparation

Data is aquired and prepared with the wrangle.py file, and performs the following tasks:

   - Clean labor data by renaming columns (EVERY column)
   - Feature Engineer additional columns
   - Set datetime index using 'date'
   - Saves a copy of the prepared dataframe to a new .csv
   - Returns prepared df

In [3]:
df = wrangle.wrangle_labor()
df.head()

Saving to .csv


,month_num,month,year,asian_american_cnp,african_american_cnp,white_cnp,asian_american_not_in_labor_force,african_american_not_in_labor_force,white_not_in_labor_force,asian_american_civilian_labor_force,asian_american_clf_rate,asian_american_employed,employed_asian_american_unemployment_rate,asian_american_unemployed,unemployed_asian_american_unemployment_rate,all_african_american_civilian_labor_force,male_african_american_civilian_labor_force,female_african_american_civilian_labor_force,all_african_american_clf_rate,male_african_american_clf_rate,female_african_american_clf_rate,all_white_civilian_labor_force,male_white_civilian_labor_force,female_white_civilian_labor_force,all_white_clf_rate,male_white_clf_rate,female_white_clf_rate,all_african_american_employed,male_african_american_employed,female_african_american_employed,all_african_american_employment_ratio,male_african_american_employment_ratio,female_african_american_employment_ratio,all_white_employed,male_white_employed,female_white_employed,employed_all_white_employment_ratio,employed_male_white_employment_ratio,employed_female_white_employment_ratio,all_african_american_unemployed,male_african_american_unemployed,female_african_american_unemployed,all_african_american_unemployment_rate,male_african_american_unemployment_rate,female_african_american_unemployment_rate,all_white_unemployed,male_white_unemployed,female_white_unemployed,all_white_unemployment_rate,male_white_unemployment_rate,female_white_unemployment_rate,total_civ_non_population,total_not_in_labor_force,avg_clf_rate,total_unemployed_pop,avg_female_clf_rate,avg_male_clf_rate,avg_female_unemployment_rate,avg_male_unemployment_rate,avg_total_unemployment_rate
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1972-01-01,1,January,1972,0.0,14332.0,126749.0,0.0,5998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8334.0,4180.0,3498.0,58.1,76.2,50.9,75608.0,43514.0,25728.0,59.7,81.7,42.7,7367.0,3802.0,3156.0,51.4,69.3,45.9,71169.0,41418.0,24403.0,56.1,77.7,40.5,967.0,378.0,343.0,11.6,9.0,9.8,4439.0,2096.0,1325.0,5.9,4.8,5.1,141081.0,5998.0,39.27,5406.0,46.80,78.95,7.45,6.90,5.83
1972-02-01,2,February,1972,0.0,14381.0,126976.0,0.0,5943.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8438.0,4271.0,3499.0,58.7,77.7,50.7,75710.0,43502.0,25788.0,59.6,81.5,42.7,7465.0,3894.0,3188.0,51.9,70.8,46.2,71310.0,41441.0,24517.0,56.2,77.6,40.6,973.0,377.0,311.0,11.5,8.8,8.9,4400.0,2062.0,1271.0,5.8,4.7,4.9,141357.0,5943.0,39.43,5373.0,46.70,79.60,6.90,6.75,5.77
1972-03-01,3,March,1972,0.0,14401.0,127191.0,0.0,5857.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8544.0,4323.0,3533.0,59.3,78.5,51.2,76262.0,43812.0,25953.0,60.0,81.9,42.9,7626.0,3974.0,3204.0,53.0,72.1,46.4,72006.0,41849.0,24688.0,56.6,78.2,40.8,918.0,349.0,329.0,10.7,8.1,9.3,4256.0,1963.0,1265.0,5.6,4.5,4.9,141592.0,5857.0,39.77,5174.0,47.05,80.20,7.10,6.30,5.43
1972-04-01,4,April,1972,0.0,14434.0,127372.0,0.0,5952.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8483.0,4330.0,3496.0,58.8,78.3,50.6,76256.0,43821.0,25816.0,59.9,81.7,42.7,7704.0,4032.0,3236.0,53.4,72.9,46.8,72378.0,42085.0,24589.0,56.8,78.5,40.6,779.0,298.0,260.0,9.2,6.9,7.4,3878.0,1736.0,1227.0,5.1,4.0,4.8,141806.0,5952.0,39.57,4657.0,46.65,80.00,6.10,5.45,4.77
1972-05-01,5,May,1972,0.0,14486.0,127601.0,0.0,5914.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8572.0,4364.0,3542.0,59.2,78.7,51.2,76421.0,43843.0,25889.0,59.9,81.6,42.7,7775.0,4063.0,3245.0,53.7,73.3,46.9,72898.0,42319.0,24647.0,57.1,78.8,40.7,797.0,302.0,297.0,9.3,6.9,8.4,3523.0,1525.0,1242.0,4.6,3.5,4.8,142087.0,5914.0,39.70,4320.0,46.95,80.15,6.60,5.20,4.63


# Data Exploration

## Initial Questions

* Is there a noticable trend or seasonality present in the data?


* Has unemployment rate increased for one population over another?


* What gender improved their employment rate the most?


* What demographic (race, gender) has decreased the most in employment rate?


* Do we see an even distribution of employment between different dempographic over time?

-------

# Is there a noticable trend or seasonality present in the data?

In [ ]:
# visualize changes in unemployment rate over time.  Use 